In [1]:
import math
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import random

In [2]:
import keras
from keras.models import Sequential
from keras.layers import Convolution2D 
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

In [3]:
### INIT CNN ###
vector = np.array((0,0,0))
###########
### CNN ###
###########
inputshape = vector.shape
model2 = Sequential()
model2.add(Dense(12,input_dim=3,activation='relu'))
model2.add(Dense(64,activation='relu'))
#model2.add(Dense(64,activation='relu'))
#model2.add(Dense(32,activation='relu'))
model2.add(Dense(units=1,activation='sigmoid')) #min_value=0, max_value=1
model2.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
#model2.compile(loss='CategoricalCrossentropy',optimizer='adam',metrics=['accuracy'])
#model2.compile(loss='mean_squared_error',optimizer='adam',metrics=['mse'])
print("the prediction for vector is: ",model2.predict(vector.reshape(1,3)))

the prediction for vector is:  [[0.5]]


In [4]:
pole_length = 0.3 #[m]
angle=0
time_interval = 0.03 #[sec]
g=9.8 #[m/s^2]
ball_mass = 0.005 #[kg]
ball_radius = 0.025 #[m]
friction_coefficient = 0.2
v0 = 0
x0 = 0 #[m]
min_angle=-8 #[deg]
max_angle=-8 #[deg]
height,width=240,640
length = 610
image = np.ones((height, width)) * 255


sucess_time = 15 #[sec]
gamma = 0.9
wins = 0
loss = 0
x_train,y_train = [],[]
i_step_in_current_round = 0 
i_step_in_total_from_beginning = 0 
num_of_round = 0 



In [5]:
def predict2degree(degreeAsFloat):
    mydegree = np.round(degreeAsFloat[0]*18.6)
    return mydegree

In [6]:
def drawImage(ballPosition,lineAngle):
    #draw line
    image = np.ones((height, width)) * 255
    normalize_x = ballPosition
    angle = lineAngle
    x_center, y_center = int(width/2), int(height/2)
    x2 = x_center+int(length/2*math.cos(angle))
    y2 = y_center+int(length/2*math.sin(angle))
    x1,y1 = (x_center-int(length/2*math.cos(angle))),(y_center-int(length/2*math.sin(angle)))
    line_thickness = 2
    cv2.line(image, (x1, y1), (x2, y2), (0, 255, 0), thickness=line_thickness)
    
    M = (y2-y1)/(x2-x1)
    y_center_of_circle = M*normalize_x + y1 -M*x1
    
    #draw circle
    radius = 10
    center_coordinates = (int(normalize_x),int(y_center_of_circle+radius))
    color = (0, 255, 0)
    thickness = 2
    image = cv2.circle(image, center_coordinates, radius, color, thickness)
    
    
    font                   = cv2.FONT_HERSHEY_SIMPLEX
    my_left_point          = (width-30,height-30)
    my_right_point         = (30,height-30)
    fontScale              = 1
    fontColor              = (0,0,0)
    lineType               = 2
    
    if(lineAngle > 0):
        cv2.putText(image,'1',my_left_point,font,fontScale,fontColor,lineType)
    else:
        cv2.putText(image,'0',my_right_point,font,fontScale,fontColor,lineType)
    image = np.flip(image)
    

    return image

In [ ]:
degree = predict2degree(model2.predict(vector.reshape(1,3)))
if(degree>=0.5):
    degree=5
else:
    degree=-5
#print("init degree",degree)
print("starting")
num_of_round=0
gamesToPlay=200
while(num_of_round<gamesToPlay):
    x_train,y_train = [],[]
    original_y=[]
    num_of_round+=1
    print("round number:",num_of_round)
    mystack = [np.ones((height, width)) * 255]
    #print("type(degree)",type(degree))
    for i in range(0,500):
        if(random.random() < math.exp(-num_of_round/(gamesToPlay*1.5))):
            degree=random.random()
            #print("randomize degree",degree)
            if(degree>=0.5):
                degree=5
            else:
                degree=-5
        else:
            p=1
            #print("degree is not randomize for this round")
        degree = float(degree)
        #print("degree:",degree)
        if(((angle>0) and (degree<0)) or ((angle<0) and (degree>0))):
            if(degree<0):
                loopsign=-1
                degree=5
            else:
                loopsign=+1
                degree=-5
            for j in range(1,10):
                degree = degree+(1*loopsign)
                #print("degree changing",degree)
                #change_sloop_slowly
                angle=math.pi*(degree/180)
                acceleration = (3/5)*g*math.sin(angle)
                current_v = v0 + acceleration*time_interval 
                current_x = x0 + v0*time_interval + (1/2)*acceleration*(time_interval**2)
                v0 = current_v
                x0 = current_x
                normalize_x = 300-current_x*300/1.5
                #print("after",time_interval,"[sec], at angle=",angle*180/math.pi,"[deg], the ball moved to ",current_x)
                #print("angle",angle)
                if(num_of_round==gamesToPlay):
                    image = drawImage(normalize_x,angle)
                    mystack = np.concatenate((mystack,[image]),axis=0)
        else:
            for j in range(1,10):
                angle=math.pi*(degree/180)
                acceleration = (3/5)*g*math.sin(angle)
                current_v = v0 + acceleration*time_interval 
                current_x = x0 + v0*time_interval + (1/2)*acceleration*(time_interval**2)
                v0 = current_v
                x0 = current_x
                normalize_x = 300-current_x*300/1.5
                #print("after",time_interval,"[sec], at angle=",angle*180/math.pi,"[deg], the ball moved to ",current_x)
                #print("angle",angle)
                if(num_of_round==gamesToPlay):
                    image = drawImage(normalize_x,angle)
                    mystack = np.concatenate((mystack,[image]),axis=0)
            
        #print("degree for i",i,"move is:",degree)
        
        #angle=math.pi*(degree/180)
        #acceleration = (3/5)*g*math.sin(angle)
        #current_v = v0 + acceleration*time_interval 
        #current_x = x0 + v0*time_interval + (1/2)*acceleration*(time_interval**2)
        #v0 = current_v
        #x0 = current_x
        #normalize_x = 300-current_x*300/1.5
        #print("after",time_interval,"[sec], at angle=",angle*180/math.pi,"[deg], the ball moved to ",current_x)
        
        #print("angle",angle)
        #image = drawImage(normalize_x,angle)
        #mystack = np.concatenate((mystack,[image]),axis=0)
        
        vector = np.array((degree/5,normalize_x/600,current_v/10))
        model_prediction = model2.predict(vector.reshape(1,3))
        #print("model_prediction",model_prediction)
        if(model_prediction>=0.5):
            model_prediction=1
            degree=5
        else:
            model_prediction=0
            degree=-5
        #degree = predict2degree(model_prediction)

        x_train.append(vector)
        y_train.append(model_prediction)
    
        #print("vector",vector)
        #print("model_prediction:",model_prediction," <==> degree:",degree)
        
        
        
        if((normalize_x > 600) or (normalize_x < 10)):
            break
        if(i*time_interval >=sucess_time):
            break
           
    print("game ended after",i,"steps")          
    x0=((random.random()-0.5)*2)/5
    v0=((random.random()-0.5)*2)/10
    original_y = y_train.copy()
    if(i*time_interval >= sucess_time):
        print("model ended without ball falling")
    elif((normalize_x > 600) or (normalize_x < 10)):
        print("game ended becouse ball dropped")
        if(len(y_train)>=12):
            #print("y_train",y_train)
            for j in range(1,8,1): 
                y_train[len(y_train)-j] = (1-y_train[len(y_train)-j])
            #print("original_y",original_y)
        else:
            for j in range(1,int(len(y_train)*0.5),1): 
                    y_train[len(y_train)-j] = (1-y_train[len(y_train)-j])   
        for j in range(random.randrange(0,int(0.2*len(y_train)))):
            numToChange =  random.randrange(0,len(y_train))
            y_train[numToChange] = (1-y_train[numToChange])
                   
    model2.fit(x=np.vstack(x_train),y=np.vstack(y_train))
    
    

starting
round number: 1
game ended after 29 steps
game ended becouse ball dropped
1/1 [==============================] - 0s 2ms/step - loss: 0.6764 - accuracy: 0.7333
round number: 2
game ended after 36 steps
game ended becouse ball dropped
2/2 [==============================] - 0s 1ms/step - loss: 0.6772 - accuracy: 0.7297
round number: 3
game ended after 10 steps
game ended becouse ball dropped
1/1 [==============================] - 0s 1ms/step - loss: 0.6816 - accuracy: 0.7273
round number: 4
game ended after 19 steps
game ended becouse ball dropped
1/1 [==============================] - 0s 3ms/step - loss: 0.7095 - accuracy: 0.5500
round number: 5
game ended after 54 steps
game ended becouse ball dropped
2/2 [==============================] - 0s 3ms/step - loss: 0.6618 - accuracy: 0.8727
round number: 6
game ended after 24 steps
game ended becouse ball dropped
1/1 [==============================] - 0s 3ms/step - loss: 0.6741 - accuracy: 0.6400
round number: 7
game ended after 16 s

In [ ]:
import imageio; 
from IPython.display import Video; 
edges = mystack
imageio.mimwrite('test2.mp4', edges, fps=33); 
Video('test2.mp4', width=640, height=480)

In [ ]:
#imageio.mimwrite('/tmp/simulate.mp4', edges, fps=33); 


In [ ]:
#x_train,y_train = [],[]
#vector = np.array((4/8,100/600,6/10))
#degree = predict2degree(model2.predict(vector.reshape(1,3)))
#print(degree)




In [ ]:
#x_train,y_train = [],[]
#print("vector:",vector)
#print("degree:",degree)
#x_train.append(vector)
#y_train.append(-1*(degree/16+0.5))
#print(y_train)
#model2.fit(x=np.vstack(x_train),y=np.vstack(y_train))

In [ ]:
print(len(y_train))

In [ ]:
print(type(y_train))

In [ ]:
y_train

In [ ]:
original_y